# Human in the Loop

In [1]:
from dataclasses import dataclass
from enum import Enum

from agentic_patterns.core.agents import get_agent, run_agent

## The Pattern

Human-in-the-loop is a control pattern where an agent pauses execution to request human approval before performing high-impact actions. The agent serializes its intended action, presents it for review, and resumes only after receiving explicit authorization.

## Scenario: Database Operations

We simulate a database management agent that can read data freely but must request approval before modifying or deleting records.

In [2]:
class ActionType(str, Enum):
    READ = "read"
    UPDATE = "update"
    DELETE = "delete"


@dataclass
class ProposedAction:
    """An action the agent wants to perform."""

    action_type: ActionType
    table: str
    description: str
    sql: str

    def requires_approval(self) -> bool:
        return self.action_type in (ActionType.UPDATE, ActionType.DELETE)

    def __str__(self) -> str:
        return f"[{self.action_type.value.upper()}] {self.description}\nSQL: {self.sql}"

## Simulated Database

In [3]:
# In-memory "database" for demonstration
DATABASE = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com", "status": "active"},
        {"id": 2, "name": "Bob", "email": "bob@example.com", "status": "active"},
        {
            "id": 3,
            "name": "Charlie",
            "email": "charlie@old-domain.com",
            "status": "inactive",
        },
    ]
}


def execute_action(action: ProposedAction) -> str:
    """Execute an approved action on the database."""
    if action.action_type == ActionType.READ:
        return f"Query result: {DATABASE.get(action.table, [])}"
    elif action.action_type == ActionType.UPDATE:
        return f"Updated {action.table}: {action.description}"
    elif action.action_type == ActionType.DELETE:
        return f"Deleted from {action.table}: {action.description}"
    return "Unknown action"

## Planning Agent

This agent analyzes a natural language request and produces a structured action plan.

In [4]:
PLANNER_PROMPT = """You are a database operations planner. Given a user request, output the SQL operation needed.

Available tables: users (columns: id, name, email, status)

Output format (exactly):
ACTION_TYPE: read|update|delete
TABLE: table_name
DESCRIPTION: brief description of what will happen
SQL: the SQL statement

Be conservative - if a request is ambiguous about what to modify, ask for clarification instead of guessing."""

planner = get_agent(system_prompt=PLANNER_PROMPT)

In [5]:
def parse_action(response: str) -> ProposedAction | None:
    """Parse the planner's response into a ProposedAction."""
    lines = response.strip().split("\n")
    data = {}
    for line in lines:
        if ":" in line:
            key, value = line.split(":", 1)
            data[key.strip().lower()] = value.strip()

    if not all(k in data for k in ["action_type", "table", "description", "sql"]):
        return None

    try:
        action_type = ActionType(data["action_type"].lower())
    except ValueError:
        return None

    return ProposedAction(
        action_type=action_type,
        table=data["table"],
        description=data["description"],
        sql=data["sql"],
    )

## Human Approval Checkpoint

The key function that implements the human-in-the-loop pattern. It presents the proposed action and waits for human input.

In [6]:
@dataclass
class ApprovalResult:
    approved: bool
    feedback: str | None = None


def request_human_approval(action: ProposedAction) -> ApprovalResult:
    """Present the action to a human and get approval."""
    print("\n" + "=" * 60)
    print("APPROVAL REQUIRED")
    print("=" * 60)
    print("\nThe agent wants to perform the following action:\n")
    print(action)
    print("\n" + "-" * 60)

    while True:
        response = input("Approve? (yes/no/modify): ").strip().lower()
        if response in ("yes", "y"):
            return ApprovalResult(approved=True)
        elif response in ("no", "n"):
            feedback = input("Reason for rejection (optional): ").strip()
            return ApprovalResult(
                approved=False, feedback=feedback or "Rejected by user"
            )
        elif response == "modify":
            feedback = input("What changes are needed? ")
            return ApprovalResult(
                approved=False, feedback=f"Modification requested: {feedback}"
            )
        print("Please enter 'yes', 'no', or 'modify'")

## Orchestrator: The Full Loop

This function ties everything together: plan the action, check if approval is needed, get approval, execute.

In [7]:
async def process_request(user_request: str) -> str:
    """Process a user request with human-in-the-loop for dangerous operations."""
    print(f"\nUser request: {user_request}")
    print("-" * 40)

    # Step 1: Plan the action
    plan_run, _ = await run_agent(planner, user_request)
    response = plan_run.result.output
    action = parse_action(response)

    if action is None:
        return f"Could not parse action from planner response:\n{response}"

    print(f"\nPlanned action: {action.action_type.value}")

    # Step 2: Check if approval is needed
    if action.requires_approval():
        approval = request_human_approval(action)

        if not approval.approved:
            return f"Action aborted. {approval.feedback}"

        print("\nApproval granted. Executing...")
    else:
        print("\nNo approval needed for read operations. Executing...")

    # Step 3: Execute the action
    result = execute_action(action)
    return result

## Example 1: Read Operation (No Approval Needed)

Read operations execute immediately without human intervention.

In [8]:
result = await process_request("Show me all users in the database")
print(f"\nResult: {result}")


User request: Show me all users in the database
----------------------------------------

Planned action: read

No approval needed for read operations. Executing...

Result: Query result: [{'id': 1, 'name': 'Alice', 'email': 'alice@example.com', 'status': 'active'}, {'id': 2, 'name': 'Bob', 'email': 'bob@example.com', 'status': 'active'}, {'id': 3, 'name': 'Charlie', 'email': 'charlie@old-domain.com', 'status': 'inactive'}]


## Example 2: Delete Operation (Approval Required)

This will pause and ask for human approval before executing.

In [9]:
result = await process_request("Remove all inactive users from the database")
print(f"\nResult: {result}")


User request: Remove all inactive users from the database
----------------------------------------

Planned action: delete

APPROVAL REQUIRED

The agent wants to perform the following action:

[DELETE] Delete all users with status 'inactive' from the users table
SQL: DELETE FROM users WHERE status = 'inactive';

------------------------------------------------------------
Please enter 'yes', 'no', or 'modify'

Approval granted. Executing...

Result: Deleted from users: Delete all users with status 'inactive' from the users table


## Example 3: Update Operation (Approval Required)

Updates also require approval before execution.

In [10]:
result = await process_request("Update Charlie's email to charlie@new-domain.com")
print(f"\nResult: {result}")


User request: Update Charlie's email to charlie@new-domain.com
----------------------------------------

Planned action: update

APPROVAL REQUIRED

The agent wants to perform the following action:

[UPDATE] Update Charlie's email address to charlie@new-domain.com
SQL: UPDATE users SET email = 'charlie@new-domain.com' WHERE name = 'Charlie';

------------------------------------------------------------

Approval granted. Executing...

Result: Updated users: Update Charlie's email address to charlie@new-domain.com
